In [ ]:
#%pip install chromadb sentence-transformers

In [11]:
import json
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
import torch

# Variables
file_path="Ecommerce_FAQ_Chatbot_dataset.json"
device = "cuda" if torch.cuda.is_available() else "cpu"
#embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2") # Embedding model
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en",model_kwargs={"device": device})
#embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device": "cuda"})
#embedding_model = HuggingFaceEmbeddings(model_name="intfloat/e5-mistral-7b-instruct",model_kwargs={"device": "cuda"})


# 📌 Load_dataset
def load_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file).get("questions", [])

# 📌 Embedding model and vector_db
def initialize_resources():
    db_client = chromadb.PersistentClient(path="chroma_db")
    db_client.delete_collection(name="faq_ecommerce")  # Elimina la colección existente
    collection = db_client.get_or_create_collection(name="faq_ecommerce")
    return collection

# 📌 Store questions and answers
def store_in_chromadb(collection, qa_pairs, model):
    if not qa_pairs:
        print("No se encontraron preguntas en el archivo JSON.")
        return
    
    existing_records = collection.count()
    print(f"🔹 Registros actuales en ChromaDB: {existing_records}")


    questions = [item.get("question", "") for item in qa_pairs]
    answers = [item.get("answer", "") for item in qa_pairs]
    #embeddings = model.encode(questions, convert_to_numpy=True).tolist()
    embeddings = model.embed_documents(questions)
    
    #collection.delete(ids=[str(i) for i in range(collection.count())])

    collection.add(
        ids=[str(i) for i in range(len(questions))],
        embeddings=embeddings,
        metadatas=[{"question": q, "answer": a} for q, a in zip(questions, answers)]
    )
    print(f"Total de registros en ChromaDB: {collection.count()}")


qa_pairs = load_dataset(file_path)
faq_collection = initialize_resources()
store_in_chromadb(faq_collection, qa_pairs, embedding_model)

🔹 Registros actuales en ChromaDB: 0
Total de registros en ChromaDB: 79


In [ ]:
# 📌 Search answers in ChromaDB
def search_faq(collection, query, model, top_k=1):
    query_embedding = model.encode(query, convert_to_numpy=True).tolist()
    
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    if results["ids"]:
        best_match = results["metadatas"][0][0]  # Get the top result
        #print(results["metadatas"][0][0])
        return best_match["answer"]
    else:
        return "Sorry, I couldn't find an answer to your question."

# Get Variables
#faq_collection = initialize_resources()


: 

In [4]:
# 📌 Chatbot
def chat():
    while True:
        user_query = input("Ask me a question (or type 'exit' to quit): ")
        if user_query.lower() == "exit":
            break
        
        response = search_faq(faq_collection, user_query, embedding_model)
        print("Chatbot:", response)


# Ejecutar el chat
if __name__ == "__main__":
    chat()

Chatbot: Wesley es el creado de este bot, pero no puedo proporcionar su numero de telefono
